In [ ]:
import numpy as np
import pandas as pd
import re
import csv
import datetime
import os

In [ ]:
## Lees alle csv-files in
bal_labels = pd.read_csv('csv_files/balanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

unbal_labels = pd.read_csv('csv_files/unbalanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

eval_labels = pd.read_csv('csv_files/eval_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

In [ ]:
## Hierin worden de wav-files gegooid
file_path = "wav_files"

In [ ]:
## Maak folder aan als deze nog niet bestaat
if not os.path.isdir(file_path):
    os.makedirs(file_path)
    
## Maak ook folders aan voor bal/unbal/eval (om later makkelijk labels weer op te halen)
if not os.path.isdir(file_path + "/bal"):
    os.makedirs(file_path + "/bal")
if not os.path.isdir(file_path + "/unbal"):
    os.makedirs(file_path + "/unbal")
if not os.path.isdir(file_path + "/eval"):
    os.makedirs(file_path + "/eval")

In [ ]:
## Geef aan of de tf-records voor jungle of urban moeten worden gedownload (ik neem aan dat we alleen jungle doen)
target = "jungle" #"urban"
mid_to_label = pd.read_csv("csv_files/class_labels_indices_" + target + ".csv", sep=";")

In [ ]:
mid_to_label

In [ ]:
def getLabels(mid_str):
    ## Maak lijst van m-id's
    mid_list = mid_str.split(',')
    labels = []
    
    ## Voor elk m-id, vind labels, (if any labels: add to label list)
    for mid in mid_list:
        if (mid_to_label.loc[mid_to_label["mid"] == mid, "mid"].any()):
            labels.append(mid_to_label.loc[mid_to_label["mid"] == mid, "index"].values[0])
    
    ## Return unique set of labels
    return set(labels)

In [ ]:
## Download wav-files van youtube
def downloadWav(vid_to_mid, folder):
    for i in range(vid_to_mid.shape[0]):
            mids = vid_to_mid.loc[i, 'positive_labels']
            labels = getLabels(mids)

            if(len(labels)>0):
                url = vid_to_mid.loc[i, 'YTID']
                start_sec = vid_to_mid.loc[i, 'start_seconds']
                start_time = str(datetime.timedelta(seconds=start_sec)) + '.00'

                cmd = 'youtube-dl -o "wav_files/' + folder + url + '_' + str(start_sec) + '00.%(ext)s" --extract-audio --youtube-skip-dash-manifest --audio-format wav --postprocessor-args "-ss ' + start_time + ' -t 00:00:10.00" "https://www.youtube.com/v/' + url + '"'

                os.system(cmd)
                print('Done with ' + url)

In [ ]:
## Hier wordt de functie aangeroepen en gaat het downloaden beginnen
## Als je de wav-files uit deze 
downloadWav(bal_labels, "bal/")
downloadWav(unbal_labels, "unbal/")
downloadWav(eval_labels, "eval/")